In [1]:
import vplanet_inference as vpi
import vplanet
import numpy as np
import pandas as pd
import scipy
import os
import astropy.units as u
import multiprocessing as mp
import tqdm

# %matplotlib widget
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
from matplotlib import rc
plt.style.use('classic')
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)
rc('figure', facecolor='w')
rc('xtick', labelsize=35)
rc('ytick', labelsize=35)

In [10]:
INFILE_DIR = "../infiles"
MB_MODEL = "matt"
EXECUTABLE = "vplanet"

outparams = {
             "final.primary.RotPer": u.day, 
             "final.primary.RotKinEnergy": u.kg*u.m**2/u.s**2,
             "final.primary.TotOrbEnergy": u.kg*u.m**2/u.s**2,
             "final.primary.LostEnergy": u.kg*u.m**2/u.s**2,
             "final.primary.RotAngMom": u.kg*u.m**2/u.s,
             "final.primary.LostAngMom": u.kg*u.m**2/u.s,
             "final.primary.DRotPerDtStellar": u.dimensionless_unscaled,
             "final.primary.DRotPerDtEqtide": u.dimensionless_unscaled,
             "final.secondary.Eccentricity": u.dimensionless_unscaled,
             "final.secondary.OrbPeriod": u.day,
             "final.secondary.RotPer": u.day,
             "final.secondary.SemiMajorAxis": u.AU,
             "final.secondary.RotKinEnergy": u.kg*u.m**2/u.s**2,
             "final.secondary.OrbEnergy": u.kg*u.m**2/u.s**2,
             "final.secondary.TotOrbEnergy": u.kg*u.m**2/u.s**2,
             "final.secondary.OrbPotEnergy": u.kg*u.m**2/u.s**2,
             "final.secondary.LostEnergy": u.kg*u.m**2/u.s**2,
             "final.secondary.RotAngMom": u.kg*u.m**2/u.s,
             "final.secondary.OrbAngMom": u.kg*u.m**2/u.s,
             "final.secondary.LostAngMom": u.kg*u.m**2/u.s,
}

# ===========================================================
# STELLAR CTL

inparams = {
            "primary.dMass": u.Msun, 
            "secondary.dMass": u.Msun, 
            "primary.dRotPeriod": u.day, 
            "secondary.dRotPeriod": u.day, 
            "primary.dTidalTau": u.dex(u.s), 
            "secondary.dTidalTau": u.dex(u.s), 
            "secondary.dEcc": u.dimensionless_unscaled, 
            "secondary.dOrbPeriod": u.day,
            "vpl.dStopTime": u.Gyr
}

inpath = os.path.join(INFILE_DIR, f"stellar_eqtide_{MB_MODEL}/ctl")
ctl = vpi.VplanetModel(inparams, inpath=inpath, outparams=outparams, timesteps=1e6*u.yr, time_init=5e6*u.yr, verbose=False, executable=EXECUTABLE)

# ===========================================================
# STELLAR CPL

inparams = {
            "primary.dMass": u.Msun, 
            "secondary.dMass": u.Msun, 
            "primary.dRotPeriod": u.day, 
            "secondary.dRotPeriod": u.day, 
            "primary.dTidalQ": u.dex(u.s), 
            "secondary.dTidalQ": u.dex(u.s), 
            "secondary.dEcc": u.dimensionless_unscaled, 
            "secondary.dOrbPeriod": u.day,
            "vpl.dStopTime": u.Gyr
}

inpath = os.path.join(INFILE_DIR, f"stellar_eqtide_{MB_MODEL}/cpl")
cpl = vpi.VplanetModel(inparams, inpath=inpath, outparams=outparams, timesteps=1e6*u.yr, time_init=5e6*u.yr, verbose=False, executable=EXECUTABLE)

In [11]:
def format_theta(tau):
    return np.array([1.0, 1.0, 0.5, 0.5, tau, tau, 0.2, 5.0, 8.0])

def plot_energy(evol, lw=2, title=None, fs=35):
    
    fig, ax = plt.subplots(1, 1, figsize=[12,12], dpi=300)
    plt.plot(evol["Time"], (evol["final.primary.LostEnergy"]+evol["final.secondary.LostEnergy"])/(1e40), color='r', linewidth=lw, label="Lost Energy")
    plt.plot(evol["Time"], (evol["final.primary.RotKinEnergy"]+evol["final.secondary.RotKinEnergy"])/(1e40), color='b', linewidth=lw, label="Rotational Kinetic Energy")
    plt.plot(evol["Time"], evol["final.secondary.OrbEnergy"]/(1e40), color='g', linewidth=lw, label="Orbital Kinetic Energy")
    plt.plot(evol["Time"], evol["final.secondary.OrbPotEnergy"]/(1e40), color='m', linewidth=lw, label="Orbital Potential Energy")

    total_energy  = (evol["final.primary.LostEnergy"]+evol["final.secondary.LostEnergy"])/(1e40)
    total_energy += (evol["final.primary.RotKinEnergy"]+evol["final.secondary.RotKinEnergy"])/(1e40)
    # total_energy += evol["final.secondary.OrbEnergy"]+evol["final.secondary.OrbPotEnergy"]
    total_energy += evol["final.secondary.TotOrbEnergy"]/(1e40)
    plt.plot(evol["Time"], total_energy, color='k', linewidth=lw, linestyle="--", label="Total Energy")

    if title is not None:
        plt.title(title, fontsize=fs)
    plt.xlabel("Time [yr]", fontsize=fs)
    plt.ylabel(r"Energy [$10^{40}\,$J]", fontsize=fs)
    plt.legend(loc='best', fontsize=fs-10, frameon=False)
    plt.xscale('log')
    plt.xlim(evol["Time"][1].value, evol["Time"][-1].value)
    plt.ylim(-4, 1)
    plt.minorticks_on()
    ax.xaxis.set_tick_params(width=3, length=10, pad=15)
    ax.yaxis.set_tick_params(width=3, length=10)
    plt.close()
    
    return fig

def plot_momentum(evol, lw=2, title=None, fs=35):
    
    fig, ax = plt.subplots(1, 1, figsize=[12,12], dpi=300)
    plt.plot(evol["Time"], evol["final.primary.LostAngMom"]+evol["final.secondary.LostAngMom"], color='r', linewidth=lw, label="Lost Angular Momentum")
    plt.plot(evol["Time"], evol["final.primary.RotAngMom"]+evol["final.secondary.RotAngMom"], color='b', linewidth=lw, label="Rotational Angular Momentum")
    plt.plot(evol["Time"], evol["final.secondary.OrbAngMom"], color='g', linewidth=lw, label="Orbital Angular Momentum")

    total_energy  = evol["final.primary.LostAngMom"]+evol["final.secondary.LostAngMom"]
    total_energy += evol["final.primary.RotAngMom"]+evol["final.secondary.RotAngMom"]
    total_energy += evol["final.secondary.OrbAngMom"]
    plt.plot(evol["Time"], total_energy, color='k', linewidth=lw, linestyle="--", label="Total Momentum")

    if title is not None:
        plt.title(title, fontsize=fs)
    plt.xlabel("Time [yr]", fontsize=fs)
    plt.ylabel(r"Angular Momentum [J$\cdot$s]", fontsize=fs)
    plt.legend(loc='best', fontsize=fs-10, frameon=False)
    plt.xscale('log')
    plt.yscale('log')
    plt.xlim(evol["Time"][1].value, evol["Time"][-1].value)
    plt.ylim(1e40, 1e46)
    plt.minorticks_on()
    ax.xaxis.set_tick_params(width=3, length=10, pad=15)
    ax.yaxis.set_tick_params(width=3, length=10)
    plt.close()
    
    return fig

In [12]:
tidal_tau = -1

evol_ctl = ctl.run_model(format_theta(tidal_tau), outsubpath="ctl_stellar", remove=False)

In [13]:
tidal_q = 6

evol_cpl = cpl.run_model(format_theta(tidal_q), outsubpath="cpl_stellar", remove=False)

In [14]:
fig = plot_energy(evol_ctl, title="STELLAR+CTL", lw=3, fs=40)
fig.savefig("../figures/energy_conservation_ctl_stellar.png", bbox_inches="tight")

In [15]:
fig = plot_energy(evol_cpl, title="STELLAR+CPL", lw=3, fs=40)
fig.savefig("../figures/energy_conservation_cpl_stellar.png", bbox_inches="tight")

In [16]:
fig = plot_momentum(evol_ctl, title="STELLAR+CTL", lw=3, fs=40)
fig.savefig("../figures/momentum_conservation_ctl_stellar.png", bbox_inches="tight")

In [17]:
fig = plot_momentum(evol_cpl, title="STELLAR+CPL", lw=3, fs=40)
fig.savefig("../figures/momentum_conservation_cpl_stellar.png", bbox_inches="tight")